In [1]:
import torch
import torchvision

from utils.data_util import *
from utils.train_util import *
from torch.utils.data import DataLoader

torch.set_printoptions(
    precision=2,
    threshold=1000,
    edgeitems=5,
    linewidth=1000,
    sci_mode=False)


In [2]:
train_dataset, test_dataset, c, h, w = get_dataset('cifar10')
net1 = LeNet5(h, w, c, num_classes=10)
model = net1.cuda()
trainloader = DataLoader(
    dataset=train_dataset,
    batch_size=160,
    shuffle=True,
    pin_memory=True,
    num_workers=8)
testloader = DataLoader(
    dataset=test_dataset,
    batch_size=160,
    shuffle=True,
    pin_memory=True,
    num_workers=8)

Files already downloaded and verified


In [3]:
loss_func = torch.nn.CrossEntropyLoss().cuda()
for i in range(10):
    trained_model = deepcopy(model)
    trained_model.train()
    optimizer = torch.optim.Adam(trained_model.parameters())
    loss_ = []
    for data, target in trainloader:
        optimizer.zero_grad()
        output = trained_model(data.cuda())
        loss = loss_func(output, target.cuda())
        loss.backward()
        optimizer.step()
        loss_.append(loss.item())
        # break

In [18]:
for data, target in testloader:
    for i in range(data.shape[0]):
        data_i = data[i, :, :]
        print(data_i.shape)
        data_i = torch.unsqueeze(data_i, 0)
        print(data_i.shape)
        output = model(data_i.cuda())
        print(output.shape, output)
        break
    break

torch.Size([3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 10]) tensor([[-0.02, -0.08, -0.09,  0.04,  0.04, -0.03,  0.00,  0.05,  0.02, -0.05]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
acc = eval_model(trained_model, testloader, 'cuda')
print(acc)

0.3333999812602997
